# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Environment
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import json
import joblib

In [7]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [8]:
ws = Workspace.from_config()
experiment = Experiment(workspace=ws, name="Detection")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-136741
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-136741


In [9]:
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "auto-ml-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)


Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [19]:
path = 'https://raw.githubusercontent.com/Tekhunt/ML-Project/master/diabetes.csv'
data = TabularDatasetFactory.from_delimited_files(path = path)
data.to_pandas_dataframe().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
   #Register Dataset in Workspace
#dataset = dataset.register(workspace=ws,
                                   #name=key,
                                   #description=description_text)



## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
# TODO: Put your automl settings here
automl_settings = {
       "n_cross_validations": 3,
       "enable_early_stopping": True,
       "experiment_timeout_minutes" :15,
       "max_concurrent_iterations": 4,
       "max_cores_per_iteration": -1,
       "verbosity": logging.INFO,
   }

# TODO: Puyour experiment name here


automl_config = AutoMLConfig(
    compute_target=cpu_cluster,
    task='classification',
    primary_metric= 'accuracy',
    training_data= data,
    label_column_name= 'Outcome',
    n_cross_validations=5)

In [18]:
# TODo your experiment name here
#remote_run = experiment.submit(automl_config)

remote_run = experiment.submit(automl_config, show_output=True)
remote_run.wait_for_completion()

Running on remote.
No run_configuration provided, running on auto-ml-cluster with default configuration
Running on remote compute: auto-ml-cluster
Parent Run ID: AutoML_5081922a-2412-4d83-a31d-77432f1c34fc

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value impu

{'runId': 'AutoML_5081922a-2412-4d83-a31d-77432f1c34fc',
 'target': 'auto-ml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-31T17:25:45.796901Z',
 'endTimeUtc': '2021-01-31T18:11:15.987692Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'auto-ml-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Detection","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-136741","workspace_name":"quick-starts-ws-136741","region":"southcentralus","compute_target":"auto-ml-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes"

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
RunDetails(remote_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [26]:
# Retrieve the best automl model.
best_run, fitted_model = remote_run.get_output()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print(best_run)
print(fitted_model)

#Register the model
description ='Fraud detection classification '
model_name='Fraud_detection'
model_path='./'
tags = None
model = remote_run.register_model(model_name = model_name, description = description , tags = tags)
print(remote_run.model_id)

Run(Experiment: Detection,
Id: AutoML_5081922a-2412-4d83-a31d-77432f1c34fc_29,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('StandardScalerWrapper',
                 <azureml...
                                    learning_rate=0.08947473684210526,
                                    max_bin=380, max_depth=6,
                                    min_child_samples=14, min_child_weight=1,
                                    min_

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [27]:
#Save the best model for the deployement
import os
os.makedirs('./amlmodel', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./amlmodel','automl_best_model_park.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./amlmodel', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [29]:
#Register the best model for the deployement

model=best_run.register_model(
            model_name = 'Fraud-Detection', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description=''
)

In [31]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [33]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_auto_park.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_auto_park.py',
                                    environment=myenv)

In [35]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [37]:
# deploye the model as a web service 
service = Model.deploy(workspace=ws, 
                       name='Fraud-Detection', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                      )# overwrite=True

WebserviceException: WebserviceException:
	Message: Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long."
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
service

In [ ]:
# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

In [ ]:
x_df=df.sample(3)
y_df = x_df.pop('')

x_df['Column1'] = 0.0

# convert the records to a json data file
recored=x_df.to_dict(orient='records')

scoring_json = json.dumps({'data': recored})
print(scoring_json)

In [ ]:

output = service.run(scoring_json)

In [ ]:
print (output)

In [ ]:
print(y_df)

In [ ]:
print(service.get_logs())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:

service.delete()
aml_compute.delete()